# 03. DeepStream with Vision model

이 강의에서는 ONNX 파일을 TensorRT 엔진 파일로 변환하여 DeepStream 애플리케이션에 배포하는 과정을 안내합니다. TensorRT는 딥러닝 모델의 추론을 최적화하여 고성능, 저지연 결과를 제공합니다.

## 03-2. DeepStream과 TensorRT 개요
- **DeepStream**: NVIDIA의 스트림 기반 비디오 분석 SDK로, TensorRT 엔진 파일을 활용하여 실시간 추론을 지원합니다.
- **TensorRT 엔진 파일**: ONNX 모델에서 변환된 최적화된 파일로, DeepStream에서 추론에 사용됩니다.

### DeepStream 애플리케이션의 기본 구성:
1. 소스 입력 (RTSP, 파일 등)
2. 추론 수행 (TensorRT용 모델 활용)
    * ONNX 모델을 TensorR용 engine파일로 변환해서 사용
3. 결과 시각화 또는 저장

## 03-3. TensorRT와 ONNX
- **TensorRT**: NVIDIA에서 개발한 고성능 딥러닝 추론 최적화 및 런타임 라이브러리.
- **ONNX (Open Neural Network Exchange)**: 다양한 프레임워크 간 상호 운용성을 지원하는 딥러닝 모델 표현 형식.

### TensorRT로 변화하는 이유는?
- DeepStream에서는 TensorRT 모델만 사용이 가능합니다.
- GPU 활용도를 극대화합니다.
- 모델 추론 지연 시간을 줄입니다.
- 추론을 위한 메모리 사용을 최적화합니다.

---

# 실습

## YOLOv8 모델로 engin파일 만들기

### 하드웨어 및 소프트웨어 요구 사항
- TensorRT를 지원하는 NVIDIA GPU.
- TensorRT SDK 설
- Python 환경 (본 실습은 Python 3.10)
- DeepStream 설치(테스트 용도, 선택 사항)

### 라이브러리 및 의존성
Python 환경에서 다음 명령어로 필요한 라이브러리를 설치합니다:
```bash
$ sudo pip3 install onnx onnxruntime tensorrt pycuda
```

### YOLOv8용 DeepStream 플러그인 설치
```bash
$ git clone https://github.com/marcoslucianops/DeepStream-Yolo.git
$ cd DeepStream-Yolo
$ make -C nvdsinfer_custom_impl_Yolo
$ mv nvdsinfer_custom_impl_Yolo/ ../
$ cd ..
$ rm -rf DeepStream-Yolo
```

### YOLOv8 pt 모델 다운로드
YOLO 라이브러리 설치 및 numpy 1.23 버전 설치

```bash
$ sudo pip3 install ultralytics
$ sudo pip3 install numpy==1.23
```

In [2]:
from ultralytics import YOLO

#YOLOv8n 모델 다운로드
model = YOLO('yolov8n')
model_path = 'yolov8n.pt'

print(f"Model downloaded and saved at: {model_path}")

Model downloaded and saved at: yolov8n.pt


In [4]:
# onnx파일로 변환
onnx_path = 'yolov8n.onnx'

model.export(format='onnx', dynamic=True, simplify=True, opset=11)  # opset=11은 TensorRT 호환을 위해 설정
print(f"ONNX model saved at: {onnx_path}")


Ultralytics 8.3.65 🚀 Python-3.10.12 torch-2.5.1 CPU (ARMv8 Processor rev 1 (v8l))


YOLOv8n summary (fused): 168 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs

PyTorch: starting from 'yolov8n.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (6.2 MB)

ONNX: starting export with onnx 1.17.0 opset 11...
ONNX: slimming with onnxslim 0.1.47...
ONNX: export success ✅ 34.0s, saved as 'yolov8n.onnx' (12.1 MB)

Export complete (36.2s)
Results saved to /home/paymentinapp/Desktop/lecture/DeepStream/강의자료/03_Vision_Model
Predict:         yolo predict task=detect model=yolov8n.onnx imgsz=640  
Validate:        yolo val task=detect model=yolov8n.onnx imgsz=640 data=coco.yaml  
Visualize:       https://netron.app
ONNX model saved at: yolov8n.onnx


### onnx 파일 확인
None이 나오면 정상

In [5]:
import onnx
model = onnx.load('yolov8n.onnx')

print(onnx.checker.check_model(model))

None


### TensorRT 빌더 스크립트 준비

TensorRT는 ONNX 모델을 엔진 파일로 변환하는 도구를 제공합니다. 아래는 Python 기반의 접근 방식입니다.

In [14]:
import tensorrt as trt

def build_engine(onnx_file_path, engine_file_path):
    # Logger 생성
    TRT_LOGGER = trt.Logger(trt.Logger.WARNING)

    # Builder, 네트워크 및 파서 생성
    builder = trt.Builder(TRT_LOGGER)
    network = builder.create_network(1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH))
    parser = trt.OnnxParser(network, TRT_LOGGER)

    # ONNX 모델 로드
    with open(onnx_file_path, "rb") as model:
        if not parser.parse(model.read()):
            print("Failed to parse the ONNX file")
            for error in range(parser.num_errors):
                print(parser.get_error(error))
            return None

    # Builder 구성
    config = builder.create_builder_config()
    config.set_memory_pool_limit(trt.MemoryPoolType.WORKSPACE, 1 << 30)  # 1GB
    config.set_flag(trt.BuilderFlag.FP16)  # FP16 활성화

    # 동적 입력 크기에 대한 최적화 프로파일 추가
    profile = builder.create_optimization_profile()
    input_name = network.get_input(0).name  # 입력 텐서 이름
    input_shape = (1, 3, 640, 640)  # 기본 YOLOv8 입력 크기
    profile.set_shape(input_name, min=(1, 3, 320, 320), opt=input_shape, max=(1, 3, 1280, 1280))
    config.add_optimization_profile(profile)

    # Engine 생성
    print("Building engine...")
    serialized_engine = builder.build_serialized_network(network, config)
    if serialized_engine:
        with open(engine_file_path, "wb") as f:
            f.write(serialized_engine)
        print(f"Engine saved to {engine_file_path}")
    else:
        print("Failed to build the engine")



# 사용 예시
onnx_path = "yolov8n.onnx"
engine_path = "yolov8n.engine"
build_engine(onnx_path, engine_path)


Building engine...
[01/31/2025-11:23:58] [TRT] [W] DLA requests all profiles have same min, max, and opt value. All dla layers are falling back to GPU
[01/31/2025-11:24:15] [TRT] [W] Tactic Device request: 200MB Available: 194MB. Device memory is insufficient to use tactic.
[01/31/2025-11:24:15] [TRT] [W] UNSUPPORTED_STATE: Skipping tactic 1 due to insufficient memory on requested size of 209715200 detected for tactic 0x000000000000001d.
[01/31/2025-11:24:15] [TRT] [W] Tactic Device request: 200MB Available: 191MB. Device memory is insufficient to use tactic.
[01/31/2025-11:24:15] [TRT] [W] UNSUPPORTED_STATE: Skipping tactic 2 due to insufficient memory on requested size of 209715200 detected for tactic 0x000000000000001e.
[01/31/2025-11:24:15] [TRT] [W] Tactic Device request: 200MB Available: 191MB. Device memory is insufficient to use tactic.
[01/31/2025-11:24:15] [TRT] [W] UNSUPPORTED_STATE: Skipping tactic 3 due to insufficient memory on requested size of 209715200 detected for tac

### 엔진 파일 테스트

`.engine` 파일이 생성되면, 올바르게 동작하는지 테스트합니다.

In [16]:
import pycuda.driver as cuda
import pycuda.autoinit
import tensorrt as trt

# TensorRT 엔진 로드
def load_engine(engine_file_path):
    TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
    with open(engine_file_path, "rb") as f:
        runtime = trt.Runtime(TRT_LOGGER)
        return runtime.deserialize_cuda_engine(f.read())

engine = load_engine("yolov8n.engine")
print("엔진이 성공적으로 로드되었습니다.")

엔진이 성공적으로 로드되었습니다.


### DeepStream에서 활용
1. **엔진 파일 배치**: `.engine` 파일을 원하는 디렉터리로 이동합니다.
2. **DeepStream 구성 업데이트**: `config_infer_primary.txt` 파일을 수정하여 새 엔진 파일을 사용하도록 설정합니다.

### **구성 파일 예제**
아래는 `config_infer_primary_yolo8n.txt`의 예제입니다:

```txt
[property]
gpu-id=0
model-engine-file=yolo8n.engine
labelfile-path=labels.txt
batch-size=1
network-mode=0
num-detected-classes=80
interval=0
gie-unique-id=1
```

아래는 config_infer_primary_yolo8n.txt를 만드는 코드입니다.

In [27]:
# config_infer_primary_yolov8.txt 파일 생성
config_content = """[property]
gpu-id=0
net-scale-factor=0.0039215697906911373
offsets=0;0;0
model-engine-file=yolov8n.engine
labelfile-path=labels.txt
batch-size=1
network-mode=2
num-detected-classes=80
interval=0
gie-unique-id=1
process-mode=1
infer-dims=3;640;640
maintain-aspect-ratio=1
output-blob-names=output0
force-implicit-batch-dim=1
uff-input-blob-name=input_1
parse-bbox-func-name=NvDsInferParseYolo
custom-lib-path=nvdsinfer_custom_impl_Yolo/libnvdsinfer_custom_impl_Yolo.so

[class-attrs-all]
threshold=0.3
eps=0.7

"""

# 파일 쓰기
with open("config_infer_primary_yolov8n.txt", "w") as f:
    f.write(config_content)

print("config_infer_primary_yolov8n.txt 파일이 생성되었습니다.")


config_infer_primary_yolov8n.txt 파일이 생성되었습니다.


COCO 데이터셋의 클래스 이름으로 label.txt 생성

In [28]:
# label.txt 생성
labels = [
    "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat",
    "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
    "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack",
    "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball",
    "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket",
    "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple",
    "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair",
    "couch", "potted plant", "bed", "dining table", "toilet", "TV", "laptop", "mouse",
    "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator",
    "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"
]

with open("labels.txt", "w") as f:
    f.write("\n".join(labels))

print("labels.txt 파일 생성 완료!")


labels.txt 파일 생성 완료!


### DeepStream 애플리케이션 코드
이제 Python을 사용하여 DeepStream 애플리케이션을 실행하는 코드를 작성합니다. 이 코드는 TensorRT 엔진 파일을 로드하고, 비디오 스트림에서 추론을 수행합니다.

In [22]:
import gi
import sys
from gi.repository import Gst, GLib

# GStreamer 초기화
gi.require_version("Gst", "1.0")
Gst.init(None)
print("GStreamer 초기화 완료")


GStreamer 초기화 완료


In [23]:
def on_message(bus, message, loop):
    """GStreamer 메시지 핸들러"""
    msg_type = message.type
    if msg_type == Gst.MessageType.EOS:
        print("End-Of-Stream 도달")
        loop.quit()
    elif msg_type == Gst.MessageType.ERROR:
        err, debug = message.parse_error()
        print(f"에러 발생: {err}, 디버그 정보: {debug}")
        loop.quit()

In [24]:
# 요소 생성
pipeline = Gst.Pipeline.new("object-detection-pipeline")

source = Gst.ElementFactory.make("v4l2src", "source")
capsfilter = Gst.ElementFactory.make("capsfilter", "capsfilter")
nvvideoconvert = Gst.ElementFactory.make("nvvideoconvert", "nvvideoconvert")
capsfilter2 = Gst.ElementFactory.make("capsfilter", "capsfilter2")
streammux = Gst.ElementFactory.make("nvstreammux", "streammux")
nvinfer = Gst.ElementFactory.make("nvinfer", "nvinfer")
nvosd = Gst.ElementFactory.make("nvdsosd", "nvosd")
sink = Gst.ElementFactory.make("nveglglessink", "sink")

if not all([source, capsfilter, nvvideoconvert, capsfilter2, streammux, nvinfer, nvosd, sink]):
    raise RuntimeError("GStreamer 요소 생성 실패")

print("요소 생성 완료")

요소 생성 완료


In [25]:
# 요소 속성 설정
source.set_property("device", "/dev/video0")
capsfilter.set_property("caps", Gst.Caps.from_string("video/x-raw, format=YUY2, width=640, height=480, framerate=30/1"))
capsfilter2.set_property("caps", Gst.Caps.from_string("video/x-raw(memory:NVMM), format=NV12, width=640, height=480, framerate=30/1"))
streammux.set_property("width", 640)
streammux.set_property("height", 480)
streammux.set_property("batch-size", 1)
streammux.set_property("batched-push-timeout", 40000)
nvinfer.set_property("config-file-path", "config_infer_primary_yolov8n.txt")

print("요소 설정 완료")

요소 설정 완료


Warn: 'threshold' parameter has been deprecated. Use 'pre-cluster-threshold' instead.


In [26]:
# 파이프라인 생성
pipeline = Gst.Pipeline.new("object-detection-pipeline")

# 파이프라인에 요소 추가
pipeline.add(source)
pipeline.add(capsfilter)
pipeline.add(nvvideoconvert)
pipeline.add(capsfilter2)
pipeline.add(streammux)
pipeline.add(nvinfer)
pipeline.add(nvosd)
pipeline.add(sink)

print("파이프라인 생성 및 요소 추가 완료")


파이프라인 생성 및 요소 추가 완료


In [28]:
# 요소 연결
try:
    if not source.link(capsfilter):
        print("source와 capsfilter 연결 실패")
    if not capsfilter.link(nvvideoconvert):
        print("capsfilter와 nvvideoconvert 연결 실패")
    if not nvvideoconvert.link(capsfilter2):
        print("nvvideoconvert와 capsfilter2 연결 실패")

    sinkpad = streammux.get_request_pad("sink_0")
    srcpad = capsfilter2.get_static_pad("src")
    if srcpad and sinkpad and not srcpad.is_linked():
        srcpad.link(sinkpad)
    else:
        print("capsfilter2와 streammux 연결 실패")

    if not streammux.link(nvinfer):
        print("streammux와 nvinfer 연결 실패")
    if not nvinfer.link(nvosd):
        print("nvinfer와 nvosd 연결 실패")
    if not nvosd.link(sink):
        print("nvosd와 sink 연결 실패")

except Exception as e:
    print(f"요소 연결 중 오류 발생: {e}")
    sys.exit(1)

In [29]:
# 파이프라인 실행
pipeline.set_state(Gst.State.PLAYING)
print("파이프라인 실행 중...")

# 메시지 루프
loop = GLib.MainLoop()
bus = pipeline.get_bus()
bus.add_signal_watch()
bus.connect("message", on_message, loop)

try:
    loop.run()
except KeyboardInterrupt:
    print("사용자 종료 요청")
finally:
    pipeline.set_state(Gst.State.NULL)
    print("파이프라인 종료 및 정리 완료")



Using winsys: x11 
Setting min object dimensions as 16x16 instead of 1x1 to support VIC compute mode.


3:27:27.191922601 21662 0xaaaacffe6da0 WARN                 nvinfer gstnvinfer.cpp:681:gst_nvinfer_logger:<nvinfer> NvDsInferContext[UID 1]: Warning from NvDsInferContextImpl::initialize() <nvdsinfer_context_impl.cpp:1243> [UID = 1]: Warning, OpenCV has been deprecated. Using NMS for clustering instead of cv::groupRectangles with topK = 20 and NMS Threshold = 0.5
3:27:27.344034248 21662 0xaaaacffe6da0 INFO                 nvinfer gstnvinfer.cpp:684:gst_nvinfer_logger:<nvinfer> NvDsInferContext[UID 1]: Info from NvDsInferContextImpl::deserializeEngineAndBackend() <nvdsinfer_context_impl.cpp:2092> [UID = 1]: deserialized trt engine from :/home/paymentinapp/Desktop/lecture/DeepStream/강의자료/03_Vision_Model/yolov8n.engine
3:27:27.344160971 21662 0xaaaacffe6da0 INFO                 nvinfer gstnvinfer.cpp:684:gst_nvinfer_logger:<nvinfer> NvDsInferContext[UID 1]: Info from NvDsInferContextImpl::generateBackendContext() <nvdsinfer_context_impl.cpp:2195> [UID = 1]: Use deserialized engine model: 

INFO: [FullDims Engine Info]: layers num: 2
0   INPUT  kFLOAT images          3x4294967295x4294967295 min: 1x3x320x320     opt: 1x3x640x640     Max: 1x3x1280x1280   
1   OUTPUT kFLOAT output0         84x4294967295   min: 0               opt: 0               Max: 0               

파이프라인 실행 중...


3:27:27.415282382 21662 0xaaaacffe6da0 INFO                 nvinfer gstnvinfer_impl.cpp:343:notifyLoadModelStatus:<nvinfer> [UID 1]: Load new model:config_infer_primary_yolov8n.txt sucessfully


에러 발생: gst-resource-error-quark: Output window was closed (3), 디버그 정보: /dvs/git/dirty/git-master_linux/3rdparty/gst/gst-nveglglessink/ext/eglgles/gsteglglessink.c(914): gst_eglglessink_event_thread (): /GstPipeline:object-detection-pipeline/GstEglGlesSink:sink
파이프라인 종료 및 정리 완료
